# <font color=orange><div align="center">SDP Project - RATP</div></font>

### <font color=orange><div align="center">19/01/2026</div></font>
### <font color=orange><div align="center">Ouissal BOUTOUATOU - Alae TAOUDI - Mohammed SBAIHI</div></font>

## 1. Lecture des données

In [1]:
import pandas as pd

data = pd.read_csv('ratp.csv')
display(data)

,peak-entering-passengers/h,peak-passing-passengers/h,off-peak-entering-passengers/h,off-peak-passing-passengers/h,"strategic priority [0,10]","Station degradation level ([0,20] scale)","connectivity index [0,100]",name
0,85000,8100,35500,3450,75,16.2,88,Odéon (Ligne 4)
1,81000,8100,37500,3150,67,17.6,95,Place d'Italie (Lign 6)
2,74000,8900,37000,4050,68,16.8,79,Jussieu (Ligne 7)
3,74000,7100,42000,4550,77,15.2,73,Nation (Ligne 9)
4,72000,7500,33000,4250,88,13.2,93,La Motte Picquet-Grenelle (Ligne 10)
5,71000,7300,31500,4600,76,15.8,93,Porte d'Orléans (Ligne 4)
6,79000,6900,39000,3800,67,16.8,79,Daumenil (Ligne 6)
7,57000,7600,40500,3800,82,17.2,77,Vaugirard (Ligne 12)
8,84000,7900,34000,3300,74,15.8,85,Oberkampf (Ligne 9)
9,72000,8700,36000,4000,66,16.6,78,Reuilly-Diderot (Ligne 1)


## 2. Tri par préférence avec somme pondérée

In [2]:
WEIGHTS = [0.021, 0.188, 0.038, 0.322, 16.124, 67.183, 16.124]

def score(row):
    s = 0
    for i, w in enumerate(WEIGHTS):
        s += row.iloc[i] * w
    return s

data['score'] = data.apply(lambda row: score(row), axis=1)

# sort by score descending
data = data.sort_values(by='score', ascending=False)
display(data[['score'] + list(data.columns[:-1])])


,score,peak-entering-passengers/h,peak-passing-passengers/h,off-peak-entering-passengers/h,off-peak-passing-passengers/h,"strategic priority [0,10]","Station degradation level ([0,20] scale)","connectivity index [0,100]",name
0,9484.2766,85000,8100,35500,3450,75,16.2,88,Odéon (Ligne 4)
1,9457.6088,81000,8100,37500,3150,67,17.6,95,Place d'Italie (Lign 6)
2,9436.2024,74000,8900,37000,4050,68,16.8,79,Jussieu (Ligne 7)
3,9389.6816,74000,7100,42000,4550,77,15.2,73,Nation (Ligne 9)
4,9349.7596,72000,7500,33000,4250,88,13.2,93,La Motte Picquet-Grenelle (Ligne 10)
5,9328.0474,71000,7300,31500,4600,76,15.8,93,Porte d'Orléans (Ligne 4)
9,9240.6938,72000,8700,36000,4000,66,16.6,78,Reuilly-Diderot (Ligne 1)
8,9229.0074,84000,7900,34000,3300,74,15.8,85,Oberkampf (Ligne 9)
6,9144.5784,79000,6900,39000,3800,67,16.8,79,Daumenil (Ligne 6)
7,9107.6636,57000,7600,40500,3800,82,17.2,77,Vaugirard (Ligne 12)


In [3]:
# ====== Helpers ==========
def get_record(data, index):
    """
    returns the record (list of attribute values) at the given index
    """
    return data.iloc[index].tolist()[:-2]


# map: record(tuple) -> name
record_dict = {}
for i in range(len(data)):
    record = get_record(data, i)
    record_dict[tuple(record)] = data.iloc[i]['name']

def get_name(record):
    return record_dict[tuple(record)]

## 3. Explicabilité entre les startions
chaque station est comparée à toutes les autres stations inférierures (en terme de score)

In [ ]:
from script import find_explanation
EXPLANATION_TYPES = ["1-m", "m-1", ["1-m", "m-1"]]

explainabilities = {} # map: record -> dict {record: list of explanation_type}
unexplainabilities = {} # map: record -> list of records


for i in range(len(data)):
    record1 = get_record(data, i)

    for j in range(i+1, len(data)):
        record2 = get_record(data, j)
        explanation_values = {get_name(record2): []}
        explanation_found = False
        for explanation_type in EXPLANATION_TYPES:
            explanation = find_explanation(record1, record2, explanation_type, verbose=False)
            if explanation_type == "1-m" and explanation and explanation["status"] != "infeasible":
                explanation_values[get_name(record2)].append(explanation_type)
                explainabilities.setdefault(get_name(record1), {}).update(explanation_values)
                explanation_found = True
            elif explanation_type == "m-1" and explanation and explanation["status"] != "infeasible":
                explanation_values[get_name(record2)] = [explanation_type]
                explainabilities.setdefault(get_name(record1), {}).update(explanation_values)
                explanation_found = True
            elif explanation_type == ["1-m", "m-1"] and explanation and explanation["status"] != "infeasible":
                explanation_values[get_name(record2)] = ["hybrid"]
                explainabilities.setdefault(get_name(record1), {}).update(explanation_values)
                explanation_found = True

        if not explanation_found:
            unexplainabilities.setdefault(get_name(record1), []).append(get_name(record2))


print("=" * 80)
i = 9
for station in explainabilities:
    print(f"Station: {station}")
    print("=" * 80)
    print(f"Explanation found for {len(explainabilities[station])}/{i} stations:")
    for explained_station in explainabilities[station]:
        explanation_types = explainabilities[station][explained_station]
        print(f"    Explains: {explained_station} via {', '.join(map(str, explanation_types))}")
    print(f"No Explanation found for {len(unexplainabilities.get(station, []))}/{i} stations")
    print("    Does not explain: ", end="")
    print(", ".join(unexplainabilities.get(station, [])))
    print("=" * 80)
    i -= 1

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2754426
Academic license 2754426 - for non-commercial use only - registered to ou___@student-cs.fr
Station: Odéon (Ligne 4)
Explanation found for 7/9 stations:
    Explains: Place d'Italie (Lign 6) via comibnation
    Explains: La Motte Picquet-Grenelle (Ligne 10) via comibnation
    Explains: Porte d'Orléans (Ligne 4) via comibnation
    Explains: Reuilly-Diderot (Ligne 1) via comibnation
    Explains: Oberkampf (Ligne 9) via comibnation
    Explains: Daumenil (Ligne 6) via comibnation
    Explains: Vaugirard (Ligne 12) via comibnation
No Explanation found for 2/9 stations
    Does not explain: Jussieu (Ligne 7), Nation (Ligne 9)
Station: Place d'Italie (Lign 6)
Explanation found for 7/8 stations:
    Explains: Nation (Ligne 9) via comibnation
    Explains: La Motte Picquet-Grenelle (Ligne 10) via comibnation
    Explains: Porte d'Orléans (Ligne 4) via comibnation
    Explains: Reuilly-Diderot (Ligne 1)